<a href="https://colab.research.google.com/github/betahikaru/kaggle-workspace/blob/master/kaggle_elo_merchant_category_recommendation_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 245kB 13.1MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c elo-merchant-category-recommendation

100% 536M/538M [00:05<00:00, 86.6MB/s]
100% 538M/538M [00:05<00:00, 97.1MB/s]
 41% 5.00M/12.2M [00:00<00:00, 42.9MB/s]
100% 12.2M/12.2M [00:00<00:00, 64.6MB/s]
 74% 36.0M/48.9M [00:00<00:00, 42.8MB/s]
100% 48.9M/48.9M [00:00<00:00, 75.3MB/s]
  0% 0.00/838k [00:00<?, ?B/s]
100% 838k/838k [00:00<00:00, 185MB/s]
  0% 0.00/1.09M [00:00<?, ?B/s]
100% 1.09M/1.09M [00:00<00:00, 195MB/s]
  0% 0.00/2.93M [00:00<?, ?B/s]
100% 2.93M/2.93M [00:00<00:00, 240MB/s]
  0% 0.00/17.2k [00:00<?, ?B/s]
100% 17.2k/17.2k [00:00<00:00, 11.6MB/s]


In [0]:
!pip install -q keras

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [0]:
test = pd.read_csv("test.csv.zip")

In [0]:
train = pd.read_csv("train.csv.zip")

In [10]:
test.describe()

,feature_1,feature_2,feature_3
count,123623.00000,123623.000000,123623.000000
mean,3.10926,1.741796,0.564377
std,1.18911,0.749195,0.495840
min,1.00000,1.000000,0.000000
25%,2.00000,1.000000,0.000000
50%,3.00000,2.000000,1.000000
75%,4.00000,2.000000,1.000000
max,5.00000,3.000000,1.000000


In [11]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [12]:
train.describe()

,feature_1,feature_2,feature_3,target
count,201917.000000,201917.000000,201917.000000,201917.000000
mean,3.105311,1.745410,0.565569,-0.393636
std,1.186160,0.751362,0.495683,3.850500
min,1.000000,1.000000,0.000000,-33.219281
25%,2.000000,1.000000,0.000000,-0.883110
50%,3.000000,2.000000,1.000000,-0.023437
75%,4.000000,2.000000,1.000000,0.765453
max,5.000000,3.000000,1.000000,17.965068


In [13]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [14]:
!pip install lightgbm

    100% |████████████████████████████████| 1.2MB 7.2MB/s 


In [0]:
import lightgbm as lgb

In [0]:
def split_year_month(df):
    new_df = df
    new_df['first_active_month'] = df['first_active_month'].fillna("0-0")
    year_month = new_df['first_active_month'].str.split("-", expand=True).astype(int)
    year_month.columns = ['year', 'month']
    new_df['year'] = year_month['year'].values
    new_df['month'] = year_month['month'].values
    new_df = new_df.drop('first_active_month', axis=1)
    print(new_df.head())
    return new_df

In [17]:
test_1 = split_year_month(test)
test_1.describe(include='all')

           card_id  feature_1  feature_2  feature_3  year  month
0  C_ID_0ab67a22ab          3          3          1  2017      4
1  C_ID_130fd0cbdd          2          3          0  2017      1
2  C_ID_b709037bc5          5          1          1  2017      8
3  C_ID_d27d835a9f          2          1          0  2017     12
4  C_ID_2b5e3df5c2          5          1          1  2015     12


,card_id,feature_1,feature_2,feature_3,year,month
count,123623,123623.00000,123623.000000,123623.000000,123623.000000,123623.000000
unique,123623,NaN,NaN,NaN,NaN,NaN
top,C_ID_9582c5bc95,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.10926,1.741796,0.564377,2016.495094,7.381709
std,NaN,1.18911,0.749195,0.495840,5.788712,3.337556
min,NaN,1.00000,1.000000,0.000000,0.000000,0.000000
25%,NaN,2.00000,1.000000,0.000000,2016.000000,5.000000
50%,NaN,3.00000,2.000000,1.000000,2017.000000,8.000000
75%,NaN,4.00000,2.000000,1.000000,2017.000000,10.000000


In [18]:
train_1 = split_year_month(train)
train_1.describe(include='all')

           card_id  feature_1  feature_2  feature_3    target  year  month
0  C_ID_92a2005557          5          2          1 -0.820283  2017      6
1  C_ID_3d0044924f          4          1          0  0.392913  2017      1
2  C_ID_d639edf6cd          2          2          0  0.688056  2016      8
3  C_ID_186d6a6901          4          3          0  0.142495  2017      9
4  C_ID_cdbd2c0db2          1          3          0 -0.159749  2017     11


,card_id,feature_1,feature_2,feature_3,target,year,month
count,201917,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000
unique,201917,NaN,NaN,NaN,NaN,NaN,NaN
top,C_ID_697c4792bd,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.105311,1.745410,0.565569,-0.393636,2016.509298,7.378745
std,NaN,1.186160,0.751362,0.495683,3.850500,0.788199,3.340718
min,NaN,1.000000,1.000000,0.000000,-33.219281,2011.000000,1.000000
25%,NaN,2.000000,1.000000,0.000000,-0.883110,2016.000000,5.000000
50%,NaN,3.000000,2.000000,1.000000,-0.023437,2017.000000,8.000000
75%,NaN,4.000000,2.000000,1.000000,0.765453,2017.000000,10.000000


In [0]:
# GridSearchCV
# https://hackkisser.hatenablog.com/entry/2018/05/22/202946
target = 'target'
predictors = ['feature_1', 'feature_2', 'feature_3']
predictors = ['feature_1', 'feature_2', 'feature_3', 'year', 'month']
bst = lgb.LGBMRegressor(
   num_leavs=31,
    learning_rate=0.01,
    min_child_samples=10,
    n_estimators=1000,
    max_depth=-1,
)

In [0]:
from sklearn.model_selection import train_test_split
# train_train, train_test = train_test_split(train, test_size=.10, random_state=1)
train_train, train_test = train_test_split(train_1, test_size=.10, random_state=1)

In [21]:
train_train.describe()

,feature_1,feature_2,feature_3,target,year,month
count,181725.000000,181725.000000,181725.000000,181725.000000,181725.000000,181725.000000
mean,3.106050,1.744922,0.565976,-0.394329,2016.509578,7.383816
std,1.187317,0.751145,0.495629,3.848273,0.787435,3.338215
min,1.000000,1.000000,0.000000,-33.219281,2011.000000,1.000000
25%,2.000000,1.000000,0.000000,-0.883014,2016.000000,5.000000
50%,3.000000,2.000000,1.000000,-0.023162,2017.000000,8.000000
75%,4.000000,2.000000,1.000000,0.763180,2017.000000,10.000000
max,5.000000,3.000000,1.000000,17.965068,2018.000000,12.000000


In [22]:
train_test.describe()

,feature_1,feature_2,feature_3,target,year,month
count,20192.000000,20192.000000,20192.000000,20192.000000,20192.000000,20192.000000
mean,3.098653,1.749802,0.561906,-0.387402,2016.506785,7.333102
std,1.175701,0.753313,0.496165,3.870571,0.795058,3.362897
min,1.000000,1.000000,0.000000,-33.219281,2012.000000,1.000000
25%,2.000000,1.000000,0.000000,-0.883473,2016.000000,5.000000
50%,3.000000,2.000000,1.000000,-0.027988,2017.000000,8.000000
75%,4.000000,2.000000,1.000000,0.787084,2017.000000,10.000000
max,5.000000,3.000000,1.000000,14.855160,2018.000000,12.000000


In [23]:
bst.fit(
    train_train[predictors],
    train_train[target],
    eval_names=['train'],
    eval_set=[
      (train_train[predictors].values, train_train[target].values)
    ],
    eval_metric='rmse',
    early_stopping_rounds=10,
    verbose=0,
)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.01, max_depth=-1,
       min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=1000, n_jobs=-1, num_leaves=31, num_leavs=31,
       objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=200000,
       subsample_freq=0)

In [24]:
from sklearn.metrics import mean_squared_error
pred1 = bst.predict(train_test[predictors])
rmse1 = np.sqrt(mean_squared_error(pred1, train_test[target]))
print(f'default RMSE: {rmse1}')

default RMSE: 3.8645710408709255


In [25]:
from sklearn.model_selection import (
    cross_val_score,
    KFold,
)
scores = -cross_val_score(
    bst,
    train_train[predictors],
    train_train[target],
    scoring='neg_mean_squared_error',
    cv=KFold(n_splits=3, shuffle=True),
)
scores = np.sqrt(scores)

print(f'Cross-Validation scores: {scores}')
print(f'Average score: {np.mean(scores)}')

Cross-Validation scores: [3.84727836 3.86168422 3.82270916]
Average score: 3.843890579995724


In [0]:
# GridSearchCV
grid_param ={
    'n_estimators': [1000,2000],
    'max_depth': [4,8,16],
    'num_leaves': [31,15,7,3],
    'learning_rate': [0.1,0.05,0.01]
}
fit_params={
    'early_stopping_rounds': 10, 
    'eval_metric' : 'rmse', 
    'eval_set' : [(train_train[predictors], train_train[target])]
}

In [27]:
train_train[:20]

,card_id,feature_1,feature_2,feature_3,target,year,month
159932,C_ID_ad05a0777d,3,1,1,0.150901,2017,9
123757,C_ID_f015a72a32,3,1,1,-0.371302,2017,11
7160,C_ID_3fdd740515,5,2,1,0.659100,2017,7
164016,C_ID_3ff438dbec,2,2,0,0.098790,2017,9
163237,C_ID_a2d4d9cc33,2,1,0,-1.352437,2017,9
138778,C_ID_e815d2f89e,2,1,0,-4.110530,2017,1
151082,C_ID_3873acdf1d,3,1,1,0.208661,2017,11
138212,C_ID_598e6f2201,5,1,1,0.154729,2015,6
54032,C_ID_ed7d821840,2,3,0,0.028941,2017,10
7222,C_ID_f987caa9d7,4,3,0,-2.807258,2017,2


In [0]:
n = 4000

In [29]:
from sklearn.model_selection import GridSearchCV
bst_gs_cv = GridSearchCV(
    bst,  # 識別器
    grid_param,  # 最適化対象パラメタ
    cv=KFold(n_splits=3, shuffle=True),
    scoring='neg_mean_squared_error',
    verbose=0,
)
bst_gs_cv.fit(
    train_train[:n][predictors],
    train_train[:n][target],
    **fit_params,
    verbose=0,
)

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
       error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.01, max_depth=-1,
       min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=1000, n_jobs=-1, num_leaves=31, num_leavs=31,
       objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=200000,
       subsample_freq=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1000, 2000], 'max_depth': [4, 8, 16], 'num_leaves': [31, 15, 7, 3], 'learning_rate': [0.1, 0.05, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [30]:
m = 20000
best_param = bst_gs_cv.best_params_
print(f'Best parameter: {best_param}')

pred2 = bst_gs_cv.predict(train_test[:m][predictors])
rmse2 = np.sqrt(mean_squared_error(pred2, train_test[:m][target]))
print(f'GridSearchCV RMSE: {rmse2}')

Best parameter: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 1000, 'num_leaves': 3}
GridSearchCV RMSE: 3.8742855921356054


In [31]:
type(pred2)

numpy.ndarray

In [32]:
pred_submit = bst_gs_cv.predict(test[predictors])
pred_submit

array([-0.67277689, -0.70154113, -0.25162513, ..., -0.59111708,
       -0.28038937, -0.59111708])

In [33]:
submit_gs_cv = test
submit_gs_cv = submit_gs_cv.drop(['first_active_month', 'feature_1', 'feature_2', 'feature_3', 'year', 'month'], axis=1)
submit_gs_cv['target'] = pred_submit
submit_gs_cv.describe()

,target
count,123623.000000
mean,-0.428392
std,0.176960
min,-0.994926
25%,-0.591117
50%,-0.473060
75%,-0.253816
max,-0.225052


In [0]:
submit_gs_cv.to_csv("submission_gs_cv.csv", index=False, header=True)

In [35]:
submit_gs_cv.head()

,card_id,target
0,C_ID_0ab67a22ab,-0.672777
1,C_ID_130fd0cbdd,-0.701541
2,C_ID_b709037bc5,-0.251625
3,C_ID_d27d835a9f,-0.253816
4,C_ID_2b5e3df5c2,-0.596821


In [36]:
submit_gs_cv.columns

Index(['card_id', 'target'], dtype='object')

In [37]:
!kaggle competitions submit -c elo-merchant-category-recommendation -f "submission_gs_cv.csv" -m "GridSearchCV, Only feature_1/2/3."

100% 4.25M/4.25M [00:00<00:00, 9.28MB/s]
Successfully submitted to Elo Merchant Category Recommendation